In [4]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup
import concurrent.futures
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
title_basic_data = pd.read_csv('./title.basics.tsv/data.tsv', sep='\t')

C:\Users\Rohit\AppData\Local\Temp\ipykernel_28164\393725127.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basic_data = pd.read_csv('./title.basics.tsv/data.tsv', sep='\t')


In [2]:
title_akas_data = pd.read_csv('./title.akas.tsv/data.tsv', sep='\t')

C:\Users\Rohit\AppData\Local\Temp\ipykernel_25396\511126621.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas_data = pd.read_csv('./title.akas.tsv/data.tsv', sep='\t')


In [33]:
print(fil_akas.region.value_counts().head(50))

region
XWW     180644
CA       89235
IN       58553
PH       31238
SG       29018
ZA       25781
HK       15869
ID       14047
JP       10468
EG        9082
TH        8052
IL        7456
IE        6402
XEU       6322
NZ        6320
US        3543
AU        2251
XAS       2036
GB        1226
BE        1199
MY         788
CH         646
NG         616
BD         567
JM         524
PK         341
DE          38
CN          31
FR          23
TW          19
ES          15
IT          15
BR          11
KR           9
SE           7
IR           6
NO           5
AE           4
SUHH         4
MX           4
DK           3
RO           3
AF           3
SI           2
PT           2
CZ           2
AT           2
TR           2
XNA          2
UA           2
Name: count, dtype: int64


In [49]:
list(fil_akas[(fil_akas.region == 'CA') | (fil_akas.region == 'US')]['titleId'].unique()).index('tt0120338')

20283

#### Filtered only English language titles from akas file

In [19]:
fil_akas = title_akas_data[title_akas_data['language'] == 'en' ]

#### Filtered only US and CA region to avoid regional movies

In [50]:
fil_akas = fil_akas[(fil_akas.region == 'CA') | (fil_akas.region == 'US')]

In [52]:
fil_akas.describe(include='all')

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
count,92778,92778.000000,92778,92778,92778,92778,92778,92778
unique,89994,NaN,82715,2,1,8,87,2
top,tt0185121,NaN,Beauty and the Beast,CA,en,imdbDisplay,\N,0
freq,9,NaN,19,89235,92778,83971,90648,64050
mean,NaN,9.044289,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,9.704543,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
unique_en_movie_ids = pd.DataFrame(list(set(fil_akas['titleId'])), columns=['tconsts_en'])
unique_en_movie_ids.to_csv("unique_en_movie_ids.csv", index=False)

In [8]:
unique_en_movie_ids = pd.read_csv("unique_en_movie_ids.csv")

In [6]:
len(set(unique_en_movie_ids))

89994

In [3]:
title_basic_data_ori = title_basic_data.copy()

In [15]:
title_basic_data.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

#### cross refer the en tconsts and keep only those titles

In [9]:
title_basic_data_en = title_basic_data[title_basic_data['tconst'].isin(unique_en_movie_ids['tconsts_en'])]

In [17]:
title_basic_data_en_30yr.describe(include='all')

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,36804,36804,36804,36804,36804.0,36804.000000,36804,36804,36804
unique,36804,1,34682,35408,3.0,NaN,1,244,734
top,tt0142239,movie,Locked In,Home,0.0,NaN,\N,90,Drama
freq,1,36804,6,7,36626.0,NaN,36804,1774,4471
mean,NaN,NaN,NaN,NaN,NaN,2014.566134,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,7.860231,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1994.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2010.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,2017.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,2021.000000,NaN,NaN,NaN


#### Keep only movies and those that have valid year

In [11]:
title_basic_data_en = title_basic_data_en[title_basic_data_en['titleType'] == 'movie']
title_basic_data_en = title_basic_data_en[title_basic_data_en['startYear'] != "\\N"]

In [13]:
title_basic_data_en.sort_values(by='startYear', axis = 0, inplace=True)

#### Keep only movies from the past. Future movies will not have box office collections

In [14]:
title_basic_data_en['startYear'] = title_basic_data_en['startYear'].astype(int)
title_basic_data_en = title_basic_data_en[title_basic_data_en['startYear'] <= 2023]

#### Filtering on only the last 30 years

In [16]:
title_basic_data_en_30yr = title_basic_data_en[title_basic_data_en['startYear']>1993]

In [22]:
title_basic_data_en_70290yr = title_basic_data_en[(title_basic_data_en['startYear']<=1993) & (title_basic_data_en['startYear']>=1970)]

In [23]:
len(title_basic_data_en_70290yr)

8549

In [ ]:
title_basic_data_en_70290yr.to_csv("title_basic_data_en_70290yr.csv", index=False)

#### Taking SNAPSHOT Cell

In [18]:
title_basic_data_en_30yr.to_csv("title_basic_data_en_30yr.csv", index=False)

### Web Scrapping


In [23]:
def scrape_movie_data(tconst):
    try:
        worldwide_collection = ""
        distributor = ""
        budget = ""
        response = requests.get(f"https://www.boxofficemojo.com/title/{tconst}")
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        perf_summ_table = soup.find('div', class_='mojo-performance-summary-table')
        worldwide_collections = perf_summ_table.find_all('span', string=re.compile(r'.Worldwide*'))
        if worldwide_collections:
            worldwide_collection = worldwide_collections[0].parent.find_all('span', class_="money")[0].text
        perf_table = soup.find('div', class_='mojo-summary-values')
        if perf_table:
            budgets = perf_table.find_all('span', string=re.compile(r'Budget'))
            if budgets:
                budget = budgets[0].next_sibling.text

            distributors = perf_table.find_all('span', string=re.compile(r'.Distributor*'))
            if distributors:
                distributor = distributors[0].next_sibling.text
                distributor = distributor[0:distributor.find("See full company information")]
        return (worldwide_collection, budget, distributor)
    except Exception as e:
        print(f"Error scraping movie {tconst}: {e}")
        return None

In [26]:
scrape_movie_data('tt8983210')

('$17,810', '', '')

In [ ]:
tconsts = title_basic_data_en['tconst'].tolist()
tconsts

In [84]:
box_offices={}
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    results = executor.map(scrape_movie_data, tconsts)

for tconst, result in zip(tconsts, results):
    box_offices[tconst] = result
    with open('box_offices.json', 'w') as file:
            json.dump(box_offices, file)

Error scraping movie tt7152802: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error scraping movie tt7153418: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error scraping movie tt20118172: 404 Client Error: Not Found for url: https://www.boxofficemojo.com/title/tt20118172/


: 

In [64]:
len(title_basic_data_en['tconst'].unique())

121480

In [ ]:
print("hello")

In [13]:
data = pd.DataFrame({
    'numerical_feature_1': [1, 2, 3, 4, 5],
    'numerical_feature_2': [10, 20, 30, 40, 50],
    'categorical_feature': ['A', 'B', 'A', 'B', 'A'],
    'target': [20, 25, 30, 35, 40]
})

# Convert categorical feature to one-hot encoding
data = pd.get_dummies(data)

X = data.drop('target', axis=1) 
y = data['target']               

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
xgb_reg.fit(X_train, y_train)

y_pred = xgb_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 4.998960494995117


In [2]:
revenue1 = pd.read_csv('./revenue_prodcompany_70290yr_withbudget.csv')
revenue2 = pd.read_csv('./revenue_prodcompany_90-latyr_withbudget.csv')

In [37]:
revenue2.describe(include="all")

,tconst,Revenue,Budget,Production_Company
count,36732,18156,2952,10628
unique,36732,17995,271,719
top,tt6632018,$713,"$30,000,000",Warner Bros.
freq,1,3,127,581


In [3]:
revenue_df =  pd.concat([revenue1, revenue2], ignore_index=True)

In [4]:
revenue_df.describe(include="all")

,tconst,Revenue,Budget,Production_Company
count,45281,20892,3216,13327
unique,45281,20671,289,812
top,tt0353824,$509,"$30,000,000",Warner Bros.
freq,1,8,134,869


In [5]:
d = revenue_df[(revenue_df['Revenue'].notna())]

In [6]:
d.describe(include="all")

,tconst,Revenue,Budget,Production_Company
count,20892,20892,3209,12802
unique,20892,20671,288,768
top,tt0106827,$509,"$30,000,000",Warner Bros.
freq,1,8,134,849


In [12]:
dt.describe(include="all")

,tconst,Revenue,Budget,Production_Company
count,12802,12802,3198,12802
unique,12802,12748,287,768
top,tt0106827,"$8,144","$30,000,000",Warner Bros.
freq,1,8,132,849


In [11]:
dt = d[(d['Production_Company'].notna())]

In [13]:
dt.to_csv("tconsts_prd_company.csv", index=False)

In [47]:
revenue_df['Production_Company'].value_counts().head(20)

Production_Company
Warner Bros.                           869
Universal Pictures                     733
Twentieth Century Fox                  596
Paramount Pictures                     594
Walt Disney Studios Motion Pictures    556
Sony Pictures Releasing                448
IFC Films                              420
Lions Gate Films                       362
Sony Pictures Classics                 334
Metro-Goldwyn-Mayer (MGM)              308
Magnolia Pictures                      303
Miramax                                294
Vertical Entertainment                 228
Columbia Pictures                      218
New Line Cinema                        211
Focus Features                         172
Fox Searchligh                         159
TriStar Pictures                       152
Kino Lorber                            132
Roadside Attractions                   132
Name: count, dtype: int64